In [3]:
from copy import deepcopy

import re

def replace_multiple_newlines(text):
    # 3つ以上の連続する改行を2つの改行に置換
    return re.sub(r'\n{2,}', '\n\n', remove_spaces_from_blank_lines(text))

def remove_spaces_from_blank_lines(text):
    # 各行を処理し、スペースのみの行を空行に置き換える
    return re.sub(r'^\s+$', '', text, flags=re.MULTILINE)

def accumulator_naive_c(accm_op, variables, indent_level=1):
    indent = " " * 4 * indent_level
    
    accm_op.append("\n")
    if len(variables) == 1:
        return variables[0]

    variables_next_ = []
    variables_ = variables[:-1] if len(variables) % 2 == 1 else variables
    for i in range(0, len(variables_), 2):
        accm_op.append(f"{indent}{variables_[i]} = {variables_[i]} + {variables_[i+1]};")
        variables_next_.append(f"{variables_[i]}")
        
    if len(variables) % 2 == 1:
        variables_next_.append(f"{variables[-1]}")
        
    accumulator_naive_c(accm_op, variables_next_, indent_level = indent_level)


def find_true_indices(lst, N):
    true_indices = [i for i, val in enumerate(lst) if val]
    return true_indices[:N]

def select_usable_var(usable_var, prefix="r", set_second=False):
    var1, var2 = "", ""
    indices = find_true_indices(usable_var, 2)
    
    var1 = f"{prefix}{indices[0]}"
    var2 = f"{prefix}{indices[1]}"
    usable_var[indices[0]] = False
    if set_second: usable_var[indices[1]] = False
    return var1, var2, usable_var


with open(f"./small_dgemv_naive_c_implementation_ver0-5_Naive-Unroll-16vars.c", "w") as p:
    p.write("#include <stddef.h>\n")
    p.write("#include <stdio.h>\n")
    p.write("#include <stdlib.h>\n")
    p.write("#include <stdint.h>\n")
    p.write("#include <math.h>\n")
    p.write("#include <omp.h>\n")
    p.write("#include <immintrin.h>\n") 
    
    for ldx in range(2, 514, 2):
        ldx_max = min([ldx, 15])

        # Declaration
        declare_x = "double " + ", ".join([f"r{ld}" for ld in range(ldx_max)])
        variables = [f"r{ld}" for ld in range(ldx_max)]

        usable_var = [True] * ldx_max

        op_concat = ""

        for ld in range(0, ldx, 2):
            var1, var2, usable_var = select_usable_var(usable_var)
            op_concat += f"""
        {var1} = a_t[{ld}];
        {var2} = a_t[{ld+1}]; 
        {var1} = x[{ld}]*{var1} + x[{ld+1}]*{var2};\n"""

            if sum(usable_var)<=1:
                accm_op = []
                accm_variables = [f"r{i}" for i in range(len(usable_var)) if not usable_var[i]]

                accumulator_naive_c(accm_op, accm_variables, indent_level=4)
                op_concat += "\n".join(accm_op)
                for var in accm_variables[1:]:
                    idx = int(var[1:])
                    usable_var[idx] = True        
        accm_op = []
        accm_variables = [f"r{i}" for i in range(len(usable_var)) if not usable_var[i]]
        accumulator_naive_c(accm_op, accm_variables, indent_level=2)
        op_concat += "\n".join(accm_op)
        base = f"""
void mydgemv_ver0_5_c_{ldx}(double a_t[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy){{
    {declare_x};

    for (int64_t i=0; i<lda; i++){{
    {op_concat}
        y[i]=r0;
        a_t += ldx;
    }}
}}
        """
    #     print(op_concat)
        p.write(replace_multiple_newlines(base))
    
    
    
    

In [7]:
for i in range(2, 514, 2):
    print(f"""
        subroutine mydgemv_ver0_5_c_{i}(a_t, x, y, lda, ldx, ldy) bind(C, name='mydgemv_ver0_5_c_{i}')
            import
            integer(c_int64_t), value :: lda, ldx, ldy
            real(c_double), intent(in) :: a_t(ldx, lda), x(ldx), y(ldy)
        end subroutine mydgemv_ver0_5_c_{i}
          """)
#     print(f"if (ldx == {i}) sub_ptr => mydgemv_ver1_c_{i}")


        subroutine mydgemv_ver0_5_c_2(a_t, x, y, lda, ldx, ldy) bind(C, name='mydgemv_ver0_5_c_2')
            import
            integer(c_int64_t), value :: lda, ldx, ldy
            real(c_double), intent(in) :: a_t(ldx, lda), x(ldx), y(ldy)
        end subroutine mydgemv_ver0_5_c_2
          

        subroutine mydgemv_ver0_5_c_4(a_t, x, y, lda, ldx, ldy) bind(C, name='mydgemv_ver0_5_c_4')
            import
            integer(c_int64_t), value :: lda, ldx, ldy
            real(c_double), intent(in) :: a_t(ldx, lda), x(ldx), y(ldy)
        end subroutine mydgemv_ver0_5_c_4
          

        subroutine mydgemv_ver0_5_c_6(a_t, x, y, lda, ldx, ldy) bind(C, name='mydgemv_ver0_5_c_6')
            import
            integer(c_int64_t), value :: lda, ldx, ldy
            real(c_double), intent(in) :: a_t(ldx, lda), x(ldx), y(ldy)
        end subroutine mydgemv_ver0_5_c_6
          

        subroutine mydgemv_ver0_5_c_8(a_t, x, y, lda, ldx, ldy) bind(C, name='mydgemv_ver0_5_c_8')
   